Q1.a Windspeed and Pressure (10 pts) Aggregate the data so that each hurricane is shown with its maximum windspeed and minimum pressure over all recorded points.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# load the data and parse "datatime"
# this makes sure the column of 'datetime' is in datetime64 format
df = pd.read_csv('hurdat2.csv', parse_dates=['datetime'])

In [3]:
# you may check the data type of the last column..
df.dtypes

identifier              object
name                    object
num_pts                  int64
record_id               object
status                  object
latitude               float64
longitude              float64
max_wind                 int64
min_pressure             int64
datetime        datetime64[ns]
dtype: object

In [4]:
df

,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime
0,AL011851,UNNAMED,14,NaN,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00
1,AL011851,UNNAMED,14,NaN,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00
2,AL011851,UNNAMED,14,NaN,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00
3,AL011851,UNNAMED,14,NaN,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00
4,AL011851,UNNAMED,14,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00
...,...,...,...,...,...,...,...,...,...,...
50298,AL192017,RINA,21,NaN,TS,38.3,-48.8,45,994,2017-11-08 12:00:00
50299,AL192017,RINA,21,NaN,TS,40.1,-49.0,45,992,2017-11-08 18:00:00
50300,AL192017,RINA,21,NaN,TS,41.8,-48.8,45,991,2017-11-09 00:00:00
50301,AL192017,RINA,21,NaN,LO,43.6,-48.0,40,993,2017-11-09 06:00:00


In [5]:
# please figure out which column you will use for "groupby"...
# your code...


agg_data = df.groupby(['name']).agg({
    'max_wind': 'max',
    'min_pressure': 'min'
}).reset_index()

In [6]:
# then you can replace values by max or min...
# think about which method you will use here...
# your code...


# Replace values in the original DataFrame
df = pd.merge(df, agg_data, on='name', how='left', suffixes=('', '_agg'))

In [7]:
# now check the value of df...
df

,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg
0,AL011851,UNNAMED,14,NaN,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00,160,-999
1,AL011851,UNNAMED,14,NaN,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00,160,-999
2,AL011851,UNNAMED,14,NaN,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00,160,-999
3,AL011851,UNNAMED,14,NaN,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00,160,-999
4,AL011851,UNNAMED,14,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00,160,-999
...,...,...,...,...,...,...,...,...,...,...,...,...
50298,AL192017,RINA,21,NaN,TS,38.3,-48.8,45,994,2017-11-08 12:00:00,100,966
50299,AL192017,RINA,21,NaN,TS,40.1,-49.0,45,992,2017-11-08 18:00:00,100,966
50300,AL192017,RINA,21,NaN,TS,41.8,-48.8,45,991,2017-11-09 00:00:00,100,966
50301,AL192017,RINA,21,NaN,LO,43.6,-48.0,40,993,2017-11-09 06:00:00,100,966


Q1.b Hour counts (10 pts) Glancing at the data, we might think that the data is reported every six hours at midnight, 6am, 12pm, and 6pm. Check to see if this is
true by calculating which hours appear in the timestamps. Which hour occurs least often?

In [8]:
# read hour and put into a new Series
s_hour = df['datetime'].dt.strftime("%H")

In [9]:
# use groupby and count for the Series s_hour
# the index is the "hour", and the value is "count"
# your code goes here...

hour_counts = s_hour.value_counts()


In [10]:
# It seems they record hours other than 00, 06, 12, 18
# the least one can be found based on "count"
# your code goes here..
least_often_hour = hour_counts.idxmin()
least_often_count = hour_counts.min()

print(f"The hour that occurs least often is {least_often_hour}, with {least_often_count} occurrences.")


The hour that occurs least often is 20, with 28 occurrences.


Q2. Speed (30 pts) Calculate the speed of the hurricane between the measured points.

In [11]:
# first provide the definition of function to compute the surface distance
import numpy as np
def haversine(lon1, lat1, lon2, lat2, earth_radius=6367):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    All args must be of equal length.
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = earth_radius * c
    return km

In [12]:
# the datetime should be grouped by id first, since the the adjacent datetime from two different huricanes will create troubles.
# your code...


# group_datetime = xxx
df = df.sort_values(by=['datetime'])

df

,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg
0,AL011851,UNNAMED,14,NaN,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00,160,-999
1,AL011851,UNNAMED,14,NaN,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00,160,-999
2,AL011851,UNNAMED,14,NaN,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00,160,-999
3,AL011851,UNNAMED,14,NaN,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00,160,-999
4,AL011851,UNNAMED,14,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00,160,-999
...,...,...,...,...,...,...,...,...,...,...,...,...
50298,AL192017,RINA,21,NaN,TS,38.3,-48.8,45,994,2017-11-08 12:00:00,100,966
50299,AL192017,RINA,21,NaN,TS,40.1,-49.0,45,992,2017-11-08 18:00:00,100,966
50300,AL192017,RINA,21,NaN,TS,41.8,-48.8,45,991,2017-11-09 00:00:00,100,966
50301,AL192017,RINA,21,NaN,LO,43.6,-48.0,40,993,2017-11-09 06:00:00,100,966


In [13]:
# Calculate time difference between consecutive points
df['time_diff'] = df.groupby('name')['datetime'].diff().dt.total_seconds() / 3600  # Convert seconds to hours

# now check the new column "time_diff"
df

,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg,time_diff
0,AL011851,UNNAMED,14,NaN,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00,160,-999,NaN
1,AL011851,UNNAMED,14,NaN,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00,160,-999,6.0
2,AL011851,UNNAMED,14,NaN,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00,160,-999,6.0
3,AL011851,UNNAMED,14,NaN,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00,160,-999,6.0
4,AL011851,UNNAMED,14,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00,160,-999,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298,AL192017,RINA,21,NaN,TS,38.3,-48.8,45,994,2017-11-08 12:00:00,100,966,6.0
50299,AL192017,RINA,21,NaN,TS,40.1,-49.0,45,992,2017-11-08 18:00:00,100,966,6.0
50300,AL192017,RINA,21,NaN,TS,41.8,-48.8,45,991,2017-11-09 00:00:00,100,966,6.0
50301,AL192017,RINA,21,NaN,LO,43.6,-48.0,40,993,2017-11-09 06:00:00,100,966,6.0


In [14]:
# we also need to group latitude and longitude for the same reason
# Calculate distance between consecutive points using haversine formula
df['distance'] = df.groupby('name').apply(
    lambda group: haversine(
        group['longitude'].shift(),
        group['latitude'].shift(),
        group['longitude'],
        group['latitude']
    )
).reset_index(level=0, drop=True)


In [15]:
# speed can be simply computed by the division between two columns.
df['speed_kph'] = df['distance'] / df['time_diff']

# Reset the index to the original identifier (row id)
df = df.reset_index(drop=True)

# now check the new column speed
df

,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg,time_diff,distance,speed_kph
0,AL011851,UNNAMED,14,NaN,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00,160,-999,NaN,NaN,NaN
1,AL011851,UNNAMED,14,NaN,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00,160,-999,6.0,58.870532,9.811755
2,AL011851,UNNAMED,14,NaN,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00,160,-999,6.0,58.870532,9.811755
3,AL011851,UNNAMED,14,NaN,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00,160,-999,6.0,50.279389,8.379898
4,AL011851,UNNAMED,14,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00,160,-999,3.0,31.424628,10.474876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298,AL192017,RINA,21,NaN,TS,38.3,-48.8,45,994,2017-11-08 12:00:00,100,966,6.0,211.322388,35.220398
50299,AL192017,RINA,21,NaN,TS,40.1,-49.0,45,992,2017-11-08 18:00:00,100,966,6.0,200.765152,33.460859
50300,AL192017,RINA,21,NaN,TS,41.8,-48.8,45,991,2017-11-09 00:00:00,100,966,6.0,189.656830,31.609472
50301,AL192017,RINA,21,NaN,LO,43.6,-48.0,40,993,2017-11-09 06:00:00,100,966,6.0,210.421792,35.070299


Q3.a Joaquin vs. Maria (10 pts) Let’s examine the speed of both Hurricane Maria and Hurricane Joaquin. Plot each hurricane’s speed over time individually using altair
as a line chart.

In [16]:
# first generate dataframe for Maria
df_maria = df[(df['datetime'].dt.strftime("%Y")=='2017') & (df['name']=='MARIA')].reset_index()
df_maria

,index,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg,time_diff,distance,speed_kph
0,50037,AL152017,MARIA,68,NaN,TD,12.2,-49.7,30,1006,2017-09-16 12:00:00,150,908,52601.5,3883.473661,0.073828
1,50039,AL152017,MARIA,68,NaN,TS,12.2,-51.7,40,1004,2017-09-16 18:00:00,150,908,6.0,217.230412,36.205069
2,50042,AL152017,MARIA,68,NaN,TS,12.4,-53.1,45,1002,2017-09-17 00:00:00,150,908,6.0,153.619969,25.603328
3,50046,AL152017,MARIA,68,NaN,TS,12.8,-54.4,55,994,2017-09-17 06:00:00,150,908,6.0,147.824307,24.637385
4,50049,AL152017,MARIA,68,NaN,TS,13.3,-55.7,60,990,2017-09-17 12:00:00,150,908,6.0,151.302381,25.217063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,50191,AL152017,MARIA,68,NaN,EX,44.9,-35.5,45,999,2017-10-01 12:00:00,150,908,6.0,352.766370,58.794395
64,50192,AL152017,MARIA,68,NaN,EX,46.5,-31.0,45,1003,2017-10-01 18:00:00,150,908,6.0,391.822153,65.303692
65,50193,AL152017,MARIA,68,NaN,EX,47.5,-26.5,40,1005,2017-10-02 00:00:00,150,908,6.0,358.627130,59.771188
66,50194,AL152017,MARIA,68,NaN,EX,48.0,-22.0,40,1012,2017-10-02 06:00:00,150,908,6.0,340.734199,56.789033


In [17]:
# you can reuse your code from assignmetn-2 to visulize
import altair as alt
alt.data_transformers.disable_max_rows()

# show the curve
chart_maria = alt.Chart(df_maria).mark_line().encode(
    x='datetime:T',
    y='speed_kph:Q',
    color='name:N'
).properties(
    title='Hurricane Maria (2017) - Speed Over Time'
)
chart_maria

alt.Chart(...)

In [18]:
# second generate the dataframe for Joaquin
df_joaquin = df[(df['datetime'].dt.strftime("%Y")=='2015') & (df['name']=='JOAQUIN')].reset_index()
df_joaquin

# Create Altair line chart for Hurricane Joaquin
chart_joaquin = alt.Chart(df_joaquin).mark_line().encode(
    x='datetime:T',
    y='speed_kph:Q',
    color='name:N'
).properties(
    title='Hurricane Joaquin (2015) - Speed Over Time'
)
chart_joaquin

alt.Chart(...)

In [19]:
# Display the charts
chart_joaquin | chart_maria


alt.HConcatChart(...)

Q3.b Superimposed Plots (10 pts) Rather than compare Joaquin and Maria side-by-side, let’s overlay them. One issue is that they have different timestamps, but we can
shift Joaquin’s dates to line up with Maria’s.

In [20]:
# align the first time stamp of maria and joaquin
# this is also suggested by the hints in A5
# to that aim, we need to genrate a "timedelta" object. Check the slides for the definition of timedelta.

# "time_shift" is an timedelta object here
time_shift = df_maria.loc[0,'datetime']- df_joaquin.loc[0,'datetime']

# then we need to add this timedelta back to the column "datetime" of joaquin
# your code...
df_joaquin['datetime'] = df_joaquin['datetime'] + time_shift


# then we drop some rows where there is "Nat"
df_joaquin_new = df_joaquin.dropna(subset = ['datetime'])

# check the new joaquin dataframe
df_joaquin_new

,index,identifier,name,num_pts,record_id,status,latitude,longitude,max_wind,min_pressure,datetime,max_wind_agg,min_pressure_agg,time_diff,distance,speed_kph
0,49006,AL112015,JOAQUIN,76,NaN,LO,26.8,-68.7,20,1011,2017-09-16 12:00:00,135,931,NaN,NaN,NaN
1,49008,AL112015,JOAQUIN,76,NaN,LO,26.9,-68.6,20,1011,2017-09-16 18:00:00,135,931,6.0,14.892449,2.482075
2,49010,AL112015,JOAQUIN,76,NaN,LO,27.0,-68.5,20,1010,2017-09-17 00:00:00,135,931,6.0,14.886609,2.481101
3,49011,AL112015,JOAQUIN,76,NaN,LO,27.1,-68.6,25,1009,2017-09-17 06:00:00,135,931,6.0,14.880751,2.480125
4,49014,AL112015,JOAQUIN,76,NaN,LO,27.2,-68.8,30,1007,2017-09-17 12:00:00,135,931,6.0,22.684445,3.780741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,49082,AL112015,JOAQUIN,76,NaN,EX,36.0,-9.0,20,1011,2017-10-03 18:00:00,135,931,6.0,67.273849,11.212308
72,49083,AL112015,JOAQUIN,76,NaN,EX,35.5,-8.7,20,1011,2017-10-04 00:00:00,135,931,6.0,61.799722,10.299954
73,49084,AL112015,JOAQUIN,76,NaN,EX,35.1,-8.4,20,1011,2017-10-04 06:00:00,135,931,6.0,52.115972,8.685995
74,49085,AL112015,JOAQUIN,76,NaN,EX,35.0,-8.0,15,1012,2017-10-04 12:00:00,135,931,6.0,38.048004,6.341334


In [21]:
# now use the shifted joaquin data for visulization
# your code... 
vis_joaquin_new = alt.Chart(df_joaquin_new).mark_line().encode(
    x='datetime:T',
    y='speed_kph:Q',
    color='name:N'
).properties(
    title='Hurricane Joaquin (Shifted) - Speed Over Time'
)


vis_joaquin_new

alt.Chart(...)

In [22]:
# visualize both dataframes
vis_maria = alt.Chart(df_maria).mark_line().encode(
    x='datetime:T',
    y='speed_kph:Q',
    color='name:N'
).properties(
    title='Hurricane Maria - Speed Over Time'
)

vis_maria | vis_joaquin_new


alt.HConcatChart(...)

Q.c. Difference Plot (5 pts)

In [23]:
# it is important to find the common time stamps for maria and joaquin (after shifting)
# we could use "merge" to find common time stamp between maria and joaquin
# we may consider "outer" options to include as much time stamps as possible
# need to indicate the common column through "lefton" and "righton"

df_diff = pd.merge(df_maria, df_joaquin_new,  left_on=["datetime"],
            right_on=["datetime"], how = "outer")

# once the new df_diff is generated, the speed difference can be computed between two "speed" columns
df_diff['speed_diff'] = df_diff['speed_kph_y'] - df_diff['speed_kph_x']

# once finishing the merge operation, we need to reset the index using reset_index(); otherwise datetime will become the index and hinder the visulization
df_diff = df_diff.reset_index()

# check your df_diff again
df_diff

,index,index_x,identifier_x,name_x,num_pts_x,record_id_x,status_x,latitude_x,longitude_x,max_wind_x,...,latitude_y,longitude_y,max_wind_y,min_pressure_y,max_wind_agg_y,min_pressure_agg_y,time_diff_y,distance_y,speed_kph_y,speed_diff
0,0,50037.0,AL152017,MARIA,68.0,NaN,TD,12.2,-49.7,30.0,...,26.8,-68.7,20.0,1011.0,135.0,931.0,NaN,NaN,NaN,NaN
1,1,50039.0,AL152017,MARIA,68.0,NaN,TS,12.2,-51.7,40.0,...,26.9,-68.6,20.0,1011.0,135.0,931.0,6.0,14.892449,2.482075,-33.722994
2,2,50042.0,AL152017,MARIA,68.0,NaN,TS,12.4,-53.1,45.0,...,27.0,-68.5,20.0,1010.0,135.0,931.0,6.0,14.886609,2.481101,-23.122227
3,3,50046.0,AL152017,MARIA,68.0,NaN,TS,12.8,-54.4,55.0,...,27.1,-68.6,25.0,1009.0,135.0,931.0,6.0,14.880751,2.480125,-22.157259
4,4,50049.0,AL152017,MARIA,68.0,NaN,TS,13.3,-55.7,60.0,...,27.2,-68.8,30.0,1007.0,135.0,931.0,6.0,22.684445,3.780741,-21.436323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.0,-9.0,20.0,1011.0,135.0,931.0,6.0,67.273849,11.212308,NaN
75,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.5,-8.7,20.0,1011.0,135.0,931.0,6.0,61.799722,10.299954,NaN
76,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.1,-8.4,20.0,1011.0,135.0,931.0,6.0,52.115972,8.685995,NaN
77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,-8.0,15.0,1012.0,135.0,931.0,6.0,38.048004,6.341334,NaN


In [24]:
# visulize the difference over time
# note three gaps can be identified in the curve, the same as shown in the A5 instruction
vis_diff = alt.Chart(df_diff).mark_line().encode(
    x='datetime:T',
    y='speed_diff:Q'
).properties(
    title='Speed Difference Between Hurricane Maria and Joaquin'
)
vis_diff

alt.Chart(...)

Q.d. Resampling (15 pts)

In [25]:
# before doing resampling, it is important to set 'datetime' as the index for both dataframes, using set_index()
# your code...

df_joaquin_resampled = df_joaquin.set_index('datetime').resample('3H').interpolate()
df_maria_resampled = df_maria.set_index('datetime').resample('3H').interpolate()

/var/folders/78/8mcy4yjn2mjdfjl_4_z0ddk80000gn/T/ipykernel_12316/3444207372.py:4: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_joaquin_resampled = df_joaquin.set_index('datetime').resample('3H').interpolate()
/var/folders/78/8mcy4yjn2mjdfjl_4_z0ddk80000gn/T/ipykernel_12316/3444207372.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_maria_resampled = df_maria.set_index('datetime').resample('3H').interpolate()


In [26]:
# after resample, we can repeat the 'merge', and generate 'speed_diff' column again
# your code....

df_diff_resampled = pd.merge(df_maria_resampled, df_joaquin_resampled, left_index=True, right_index=True, how="outer")

# Compute the speed difference between two "speed" columns
df_diff_resampled['speed_diff_resampled'] = df_diff_resampled['speed_kph_y'] - df_diff_resampled['speed_kph_x']


In [27]:
# once finished, we shall reset the index to the default one through 'reset_index()'
# your code...
df_diff_resampled = df_diff_resampled.reset_index()

# Check the df_diff_resampled
df_diff_resampled

,datetime,index_x,identifier_x,name_x,num_pts_x,record_id_x,status_x,latitude_x,longitude_x,max_wind_x,...,latitude_y,longitude_y,max_wind_y,min_pressure_y,max_wind_agg_y,min_pressure_agg_y,time_diff_y,distance_y,speed_kph_y,speed_diff_resampled
0,2017-09-16 12:00:00,50037.0,AL152017,MARIA,68.0,NaN,TD,12.2,-49.7,30.0,...,26.80,-68.70,20.0,1011.0,135.0,931.0,NaN,NaN,NaN,NaN
1,2017-09-16 15:00:00,50038.0,NaN,NaN,68.0,NaN,NaN,12.2,-50.7,35.0,...,26.85,-68.65,20.0,1011.0,135.0,931.0,NaN,NaN,NaN,NaN
2,2017-09-16 18:00:00,50039.0,AL152017,MARIA,68.0,NaN,TS,12.2,-51.7,40.0,...,26.90,-68.60,20.0,1011.0,135.0,931.0,6.0,14.892449,2.482075,-33.722994
3,2017-09-16 21:00:00,50040.5,NaN,NaN,68.0,NaN,NaN,12.3,-52.4,42.5,...,26.95,-68.55,20.0,1010.5,135.0,931.0,6.0,14.889529,2.481588,-28.422610
4,2017-09-17 00:00:00,50042.0,AL152017,MARIA,68.0,NaN,TS,12.4,-53.1,45.0,...,27.00,-68.50,20.0,1010.0,135.0,931.0,6.0,14.886609,2.481101,-23.122227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2017-10-04 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.10,-8.40,20.0,1011.0,135.0,931.0,6.0,52.115972,8.685995,NaN
143,2017-10-04 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.05,-8.20,17.5,1011.5,135.0,931.0,6.0,45.081988,7.513665,NaN
144,2017-10-04 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.00,-8.00,15.0,1012.0,135.0,931.0,6.0,38.048004,6.341334,NaN
145,2017-10-04 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.10,-7.85,15.0,1012.0,135.0,931.0,6.0,36.615761,6.102627,NaN


In [28]:
# visulize the speed difference over time
vis_diff_resampled = alt.Chart(df_diff_resampled).mark_line().encode(
    x='datetime:T',
    y='speed_diff_resampled:Q'
).properties(
    title='Speed Difference (Resampled) Between Hurricane Maria and Joaquin'
)

vis_diff_resampled

alt.Chart(...)